## Data processing

In [0]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [6]:
data = pd.read_csv('prepared_data.csv')
targets = pd.read_csv('targets.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2283) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
data.describe()

,Unnamed: 0,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGMFU,CIGREC,CIG30USE,CG30EST,CIG30AV,...,IRFAMIN3,GOVTPROG,INCOME,POVERTY3,TOOLONG,TROUBUND,PDEN10,COUTYP4,MAIIN102,AIIND102
count,48175.000000,48175.000000,48167.000000,48165.000000,47863.000000,47710.000000,48175.000000,48083.000000,48145.000000,48137.000000,...,48174.000000,48174.000000,48174.000000,47786.000000,48071.000000,48071.000000,48174.000000,48174.000000,48174.000000,48174.000000
mean,24087.000000,1.511738,78.221023,78.231454,518.146773,92.972018,47.891977,78.476197,92.980164,75.172051,...,4.838627,1.774235,2.635945,2.386933,1.936199,1.952737,1.653298,1.746959,1.980757,1.980861
std,13907.068946,0.499867,39.311267,39.295169,487.343948,13.078214,44.160047,28.145189,4.359900,34.483832,...,2.095976,0.418089,1.143432,0.794082,0.244401,0.212204,0.619325,0.764557,0.137379,0.137015
min,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,12043.500000,1.000000,99.000000,99.000000,16.000000,91.000000,3.000000,91.000000,91.000000,91.000000,...,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,2.000000,2.000000
50%,24087.000000,2.000000,99.000000,99.000000,991.000000,91.000000,91.000000,91.000000,91.000000,91.000000,...,5.000000,2.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
75%,36130.500000,2.000000,99.000000,99.000000,991.000000,99.000000,91.000000,93.000000,93.000000,93.000000,...,7.000000,2.000000,4.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
max,48174.000000,2.000000,99.000000,99.000000,991.000000,99.000000,91.000000,93.000000,99.000000,93.000000,...,7.000000,2.000000,4.000000,3.000000,2.000000,2.000000,3.000000,3.000000,2.000000,2.000000


#### Data Separation
I want to predict if a person uses particular drug using answers to the specific-drug-unrelated questions in the survey and then using questions not connected to any drug usage. For that I need to separate all drug related variables from general data.

##### Prefixes in variables-questions used for every type of drug:

In [16]:
data['CIGTRY'].unique()

array([ 16.,  15.,  26., 991.,   5.,  12.,  nan,  22.,  14.,  35.,  18.,
        19.,  17.,  21.,  10.,   9.,  11.,   7.,  13.,  25.,  23.,  28.,
        20.,   6.,  30.,   8.,  24.,  27.,  32.,  34.,   4.,  42.,  29.,
        40.,  31.,  39.,  36.,  47.,  48.,   3.,  33.,  38.,  45.,  60.,
        44.,   2.,  46.,  37.,   1.,  50.,  43.,  51.,  55.,  61.])

In [0]:
halucinogens = ['LSD', 'PCP','PEYOTE','MESC','PSILCY','ECST','KET','DMTAMTFXY','SALVIADIV']
inhalants = ['AMYLNIT', 'CLEFLU', 'GAS', 'GLUE', 'ETHER', 'SOLVENT', 'LGAS', 'NITOXID',
             'FELTMARKR', 'SPPAINT', 'AIRDUSTER', 'OTHAEROS']

In [0]:
prefixes = {}
prefixes['tobacco'] = 'CIG', 'CGR', 'CI3', 'PIP', 'SMK', 'TOB', 'CD'
prefixes['alcohol'] = ('AL')
prefixes['marijuana'] = 'MJ', 'MR'
prefixes['cocaine'] = 'COC', 'CC', 'CON'
prefixes['crack'] = 'CRK', 'CR' 
prefixes['heroin'] = 'HER', 'HR', 'HEO'
prefixes['hallucinogens'] = ['HAL']+halucinogens
prefixes['inhalants'] = ['INH']+inhalants
prefixes['methamphetamine'] = 'MET', 'ME'
prefixes['pain relievers'] = 'PNR', 'OXC'
prefixes['tranqualizers'] = ('TRQ')
prefixes['stimulants'] = 'SED', 'PSY'
prefixes['sedaives'] = ('STM')
prefixes['special drugs'] = 'COLD', 'GH', 'OT'
prefixes['general about drugs'] = ('GNN')

In [10]:
drug_cols = []
for drug in prefixes.keys():
  drug_cols += [col for col in data.columns if col.startswith(tuple(prefixes[drug]))]
len(drug_cols)

1927

#### Dealing with missing values
Due to the cpecifics of the survey data I'll use 2 types of missing value imputation. For all drug-related columns I'll use global most common substitution, because most peope do not use drugs. The rest columns with missing values will be imputed by KNN model

In [0]:
imputer = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
imputer.fit_transform(data[drug_cols])

influence of RSK

special drugs

needle any